In [21]:
from __future__ import print_function, division

import torch
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, transforms, utils
import time
import os
import copy
import torch.nn as nn
import torch.nn.functional as F
import datetime

net_param_set = [{'conv':[(),
                          (3, 16, 5, 1, 1), 
                          (16, 32, 3, 1, 1),
                          (32, 32, 3, 1, 0),
                          (32, 16, 3, 1, 1)], # in_channels, out_channels, kernel_size, stride, padding
              'pool':[(), 
                      (2, 2, 0),
                      (2, 2, 0)], # kernel_size, stride, padding
              'fc':[(), 
                    (16*6*6, 120),
                    (120, 90), 
                    (90, 10)] # in_channels, out_channels
             },
             {'conv':[(), 
                      (3, 32, 5, 1, 1), 
                      (32, 32, 3, 1, 0),
                      (32, 16, 3, 1, 1)], # in_channels, out_channels, kernel_size, stride, padding
              'pool':[(), 
                      (2, 2, 0),
                      (2, 2, 0)], # kernel_size, stride, padding
              'fc':[(), 
                    (16*6*6, 120),
                    (120, 90), 
                    (90, 10)] # in_channels, out_channels
             },
             {'conv':[(), 
                      (3, 32, 5, 1, 1), 
                      (32, 16, 3, 1, 0)], # in_channels, out_channels, kernel_size, stride, padding
              'pool':[(), 
                      (2, 2, 0),
                      (2, 2, 0)], # kernel_size, stride, padding
              'fc':[(), 
                    (16*6*6, 120),
                    (120, 90), 
                    (90, 10)] # in_channels, out_channels
             }]
             
epoch_max = 30

optimizer_param_set = [(1e-4, 'expo'),
                       (5e-4, 'expo'),
                       (1e-5, 'step'), 
                       (5e-5, 'step'), 
                       (1e-4, 'step'), 
                       (5e-4, 'step'),
                       (1e-3, 'step')] # learning rate, decay strategy 

In [22]:
def test(model):
    
    # overall training correct rate
    correct = 0
    total = 0
    with torch.no_grad():
        for i, data in enumerate(dataloaders['train'], 0):
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Training accuracy: %d %%' % (100 * correct / total))
    
    # overall testing correct rate
    correct = 0
    total = 0
    with torch.no_grad():
        for i, data in enumerate(dataloaders['test'], 0):
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Testing accuracy: %d %%' % (100 * correct / total))
    
    # count testing predictions for each class
    correct_pred = {classname: 0 for classname in class_names}
    total_pred = {classname: 0 for classname in class_names}
    with torch.no_grad():
        for i, data in enumerate(dataloaders['test'], 0):
            images, labels = data
            outputs = model(images)
            _, predictions = torch.max(outputs, 1)
            for label, prediction in zip(labels, predictions):
                if label == prediction:
                    correct_pred[class_names[label]] += 1
                total_pred[class_names[label]] += 1

    # print accuracy for each class
    print("Testing accuracy (each class): ")
    for classname, correct_count in correct_pred.items():
        accuracy = 100 * float(correct_count) / total_pred[classname]
        print("{:1s}: {:.1f}%;  ".format(classname, accuracy), end=' ')
        if classname == "5":
            print()
    print()
    
    return


def train_test(model, criterion, optimizer, scheduler, num_epochs=25):
    
    for epoch in range(num_epochs):  

        running_loss = 0.0
        loss_record=[]
        for i, data in enumerate(dataloaders['train'], 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                loss_record.append(round(running_loss / 2000, 4))
                # print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0
                
        print(datetime.datetime.now(), ' Epoch', (epoch + 1), ': Average Loss', loss_record)

    print('Finished Training')
    
    test(model)
    
    return None

# Data transformer
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((32,32)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((32,32)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Dataset initialization
data_dir = 'data' 
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'test']} # Read train and test sets, respectively.

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=0) for x in ['train', 'test']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}

class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # Set device to "cpu" if you have no gpu


In [33]:
class Net(nn.Module):
    """
    Input - 1x32x32
    Output - 10
    CONV1->CONV2->POOL1->CONV3->CONV4->POOL2->FC1->FC2->FC3
    """
    def __init__(self, params):
        super(Net, self).__init__()
        
        #Initialize layers
        self.params = params
        self.n_layers = len(params['conv'])+len(params['pool'])+len(params['fc'])-3
        self.printed = False
        
        if self.n_layers == 9:
            self.conv1 = nn.Conv2d(*self.params['conv'][1])
            self.conv2 = nn.Conv2d(*self.params['conv'][2])
            self.conv3 = nn.Conv2d(*self.params['conv'][3])
            self.conv4 = nn.Conv2d(*self.params['conv'][4])
            
            self.pool1 = nn.MaxPool2d(*self.params['pool'][1])
            self.pool2 = nn.MaxPool2d(*self.params['pool'][2])
            
            self.fc1 = nn.Linear(*self.params['fc'][1])
            self.fc2 = nn.Linear(*self.params['fc'][2])
            self.fc3 = nn.Linear(*self.params['fc'][3])
            
        elif self.n_layers == 8:
            self.conv1 = nn.Conv2d(*self.params['conv'][1])
            self.conv2 = nn.Conv2d(*self.params['conv'][2])
            self.conv3 = nn.Conv2d(*self.params['conv'][3])
            
            self.pool1 = nn.MaxPool2d(*self.params['pool'][1])
            self.pool2 = nn.MaxPool2d(*self.params['pool'][2])
            
            self.fc1 = nn.Linear(*self.params['fc'][1])
            self.fc2 = nn.Linear(*self.params['fc'][2])
            self.fc3 = nn.Linear(*self.params['fc'][3])
        
        elif self.n_layers == 7:
            self.conv1 = nn.Conv2d(*self.params['conv'][1])
            self.conv2 = nn.Conv2d(*self.params['conv'][2])
            
            self.pool1 = nn.MaxPool2d(*self.params['pool'][1])
            self.pool2 = nn.MaxPool2d(*self.params['pool'][2])
            
            self.fc1 = nn.Linear(*self.params['fc'][1])
            self.fc2 = nn.Linear(*self.params['fc'][2])
            self.fc3 = nn.Linear(*self.params['fc'][3])
            

    def forward(self, img):
        # Implement forward pass
        x = img
        
        if self.n_layers == 9:
            x = F.relu(self.conv1(x))
            if not self.printed: 
                print("CONV1", x.size(), end=" || ")
            x = F.relu(self.conv2(x))
            if not self.printed: 
                print("CONV2", x.size(), end=" || ")
            x = self.pool1(x)
            if not self.printed: 
                print("POOL1", x.size())
        
            x = F.relu(self.conv3(x))
            if not self.printed: 
                print("CONV3", x.size(), end=" || ")
            x = F.relu(self.conv4(x))
            if not self.printed: 
                print("CONV4", x.size(), end=" || ")
            x = self.pool2(x)
            if not self.printed: 
                print("POOL2", x.size())
        
            x = x.view(x.size(0), -1)
            x = F.relu(self.fc1(x))
            if not self.printed: 
                print("FC1", x.size(), end=" || ")
            x = F.relu(self.fc2(x))
            if not self.printed: 
                print("FC2", x.size(), end=" || ")
            x = self.fc3(x)
            if not self.printed: 
                print("FC3", x.size())
                
        elif self.n_layers == 8:
            x = F.relu(self.conv1(x))
            if not self.printed: 
                print("CONV1", x.size(), end=" || ")
            x = self.pool1(x)
            if not self.printed: 
                print("POOL1", x.size())
        
            x = F.relu(self.conv2(x))
            if not self.printed: 
                print("CONV2", x.size(), end=" || ")
            x = F.relu(self.conv3(x))
            if not self.printed: 
                print("CONV3", x.size(), end=" || ")
            x = self.pool2(x)
            if not self.printed: 
                print("POOL2", x.size())
        
            x = x.view(x.size(0), -1)
            x = F.relu(self.fc1(x))
            if not self.printed: 
                print("FC1", x.size(), end=" || ")
            x = F.relu(self.fc2(x))
            if not self.printed: 
                print("FC2", x.size(), end=" || ")
            x = self.fc3(x)
            if not self.printed: 
                print("FC3", x.size())
        
        elif self.n_layers == 7:
            x = F.relu(self.conv1(x))
            if not self.printed: 
                print("CONV1", x.size(), end=" || ")
            x = self.pool1(x)
            if not self.printed: 
                print("POOL1", x.size())
        
            x = F.relu(self.conv2(x))
            if not self.printed: 
                print("CONV2", x.size(), end=" || ")
            x = self.pool2(x)
            if not self.printed: 
                print("POOL2", x.size())
        
            x = x.view(x.size(0), -1)
            x = F.relu(self.fc1(x))
            if not self.printed: 
                print("FC1", x.size(), end=" || ")
            x = F.relu(self.fc2(x))
            if not self.printed: 
                print("FC2", x.size(), end=" || ")
            x = self.fc3(x)
            if not self.printed: 
                print("FC3", x.size())
        
        self.printed = True

        return x


In [34]:
paths = []
for j in range(len(optimizer_param_set)):
    for i in range(len(net_param_set)):
        model_ft = Net(net_param_set[i])
        model_ft = model_ft.to(device)
        criterion = nn.CrossEntropyLoss()
        learning_rate, decay_strategy = optimizer_param_set[j]
        optimizer_ft = optim.Adam(model_ft.parameters(), lr=learning_rate)
        if decay_strategy=='expo':
            lr_scheduler = lr_scheduler.ExponentialLR(optimizer_ft, gamma=0.1)
        elif decay_strategy=='step':
            lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=20, gamma=0.1)
        else:
            print("EEOR 2")
        
        print(i, ",", j)
        for n in range((epoch_max // 5)):
            print(datetime.datetime.now())
            epo = 5*n+5
            print("epoch range: ", epo-4, " to ", epo)
            train_test(model_ft, criterion, optimizer_ft, lr_scheduler, num_epochs=5)
        
        PATH = "./cnn"+str(i)+"_"+str(j)+"_.pth"
        paths.append(PATH)
        torch.save({'epoch': epoch,
                    'model_state_dict': model_ft.state_dict(),
                    'optimizer_state_dict': optimizer_ft.state_dict()
                   }, PATH)


0 , 0
2021-05-08 17:45:15.144096
epoch range:  1  to  5
CONV1 torch.Size([4, 16, 30, 30]) || CONV2 torch.Size([4, 32, 30, 30]) || POOL1 torch.Size([4, 32, 15, 15])
CONV3 torch.Size([4, 32, 13, 13]) || CONV4 torch.Size([4, 16, 13, 13]) || POOL2 torch.Size([4, 16, 6, 6])
FC1 torch.Size([4, 120]) || FC2 torch.Size([4, 90]) || FC3 torch.Size([4, 10])


KeyboardInterrupt: 